# Spotify Mood Session

## 1. Algorithm

a. **First Selection**. User selects a song $s_0$ from the set of all songs $S$ using some kind of search method (by name / artist) and plays that song.

b. **Played List**.  Let $P$ be a new list of played songs and let $P \gets s_0$   

c. **Cold Start**. Create a bootstrap random selection of $n$ songs from $S$ called $R$. Calculate the distance between each song and the first selection and select the closest to the chosen song . Formally:

$$
\forall s \in R \text{ dist}(s_i, s_0)
$$

*potential parameter: use a value to add the closest n songs to the initial bootstrap played list*

d. **Rating**. For every song that a user plays for more than 30 seconds, the song is rated $\text{pos}$, songs that are skipped within this time period are rated $\text{neg}$.  Songs that have not been played are rated $\text{non}$

e. **Recommending**.  Take the current set of vectors and select the next song based on some sort of agregated mean or nearest neighbour system.

repeat (d) and (e) ad infinitum.

Formally:

$$
\text{let: } s_0 \gets \text{ user}(S) \\
\text{let: } P = \{ s_0 \} \\
\text{let: } R \subset S \\
\text{let: } D = \emptyset \\ 
\text{for } s_i \text{ in } R:\\
D_i = \text{dist}(s_i, s_0) \\
\text{end for }\\
\text{select}(\text{max}(D))\\
$$

In [1]:
import json
import pandas as pd
import os
from IPython.display import display

# create list of playlists
playlists = []
path = 'data'
count = 5
for filename in os.listdir(path):
    count -= 1
    if count < 0:
        break
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        d = json.load(open(f))
        playlists.append(pd.DataFrame(d['playlists']))

playlists = pd.concat(playlists)
playlists = playlists.reset_index(drop=True)
print(playlists.shape)

(5000, 12)


In [2]:
# Add all songs from the playlists into a single list of songs
songs = [] 
for i in range(len(playlists)):
    tracks = playlists.iloc[i].loc['tracks']
    for track in tracks:
        songs.append(track)

print(len(songs))
print(songs[0])

334487
{'pos': 0, 'artist_name': 'Missy Elliott', 'track_uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI', 'artist_uri': 'spotify:artist:2wIVse2owClT7go1WT98tk', 'track_name': 'Lose Control (feat. Ciara & Fat Man Scoop)', 'album_uri': 'spotify:album:6vV5UrXcfyQD1wu4Qo2I9K', 'duration_ms': 226863, 'album_name': 'The Cookbook'}


In [3]:
songs_df = pd.DataFrame(songs)
print(songs_df.shape)

songs_df.head()

(334487, 8)


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot


In [4]:
# Remove duplicate songs by URI
songs_df.drop_duplicates('track_name')

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot
...,...,...,...,...,...,...,...,...
334480,13,Lenka,spotify:track:62FaMvOq3vLJTBjb5RgxiO,spotify:artist:5g3uG8zZZANGT6YOssgjfC,Like A Song,spotify:album:2n2VzyRpAty4hkMHBx1nV0,200173,Lenka
334482,15,Jay Ungar,spotify:track:5YUeOSawPzseEvCvi1HbrE,spotify:artist:54jNT6jMGDoxMSBzUDEYSB,Cows On The Hill,spotify:album:38D9S6sriWlYlbUBONInpa,225440,"Waltzing With You (Music From The Film ""Brothe..."
334483,16,Solas,spotify:track:4aAmDp1AR1Fp1UCpWuvy2d,spotify:artist:2CGGUtRYybRjQmuUwXugeP,Crested Hens,spotify:album:1EL10LDAWFZdencyTUlGqu,314186,Solas
334484,17,Kim Yoon,spotify:track:25JRhSij3kxAJ1li9lPe5k,spotify:artist:2wA50FpX5Yc5b4OrhdQVXP,Moonlight Waltz,spotify:album:6pFFE9TLlHpeQrvOVNiswy,226925,Dreaming in the Blue Sky


In [5]:
# Add score placeholder to the songs
classification = [0]*len(songs_df)
songs_df.insert(8, "class", classification) # Append data frame by one column

In [6]:
# Print song data to ensure the score was a
print(songs_df.iloc[0])


pos                                                     0
artist_name                                 Missy Elliott
track_uri            spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
artist_uri          spotify:artist:2wIVse2owClT7go1WT98tk
track_name     Lose Control (feat. Ciara & Fat Man Scoop)
album_uri            spotify:album:6vV5UrXcfyQD1wu4Qo2I9K
duration_ms                                        226863
album_name                                   The Cookbook
class                                                   0
Name: 0, dtype: object


In [7]:
# Search for a particular song 
print("Search for a song name")
s = input()
pattern = [s]

#filter for rows that contain the partial string "Wes" in the conference column
results = songs_df.copy()
results[songs_df.track_name.str.contains('|'.join(pattern))]

Search for a song name


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,class
76,24,Queen,spotify:track:1AhDOtG9vPSOmsWgNW0BEY,spotify:artist:1dfeR4HaWDbWqFHLkxsg1d,Bohemian Rhapsody - Remastered 2011,spotify:album:1TSZDcvlPtAnekTaItI3qO,354320,A Night At The Opera,0
132,41,G-DRAGON,spotify:track:5FxasBmho2yEvgXrzp9MMW,spotify:artist:30b9WulBM8sFuBo17nNq9c,"Untitled, 2014 - KR Ver.",spotify:album:3TttE8nRMlr9zDewnUbu0c,222867,KWON JI YONG,0
136,45,AKDONG MUSICIAN,spotify:track:23DBqqe35rEZOKJk0qYPVy,spotify:artist:6OwKE9Ez6ALxpTaKcT5ayv,200%,spotify:album:03eUMhwODEJRYsYEiqtOpo,193321,Play,0
138,47,BTS,spotify:track:6RioKpKPuG1vuyU0wnlAZx,spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX,21st Century Girl,spotify:album:17FnTn4P3Bkyf6mbNQDhhy,192596,Wings,0
156,1,Camille Saint-Saëns,spotify:track:1Kzxd1kkjaGX4JZz2CYsXB,spotify:artist:436sYg6CZhNefQJogaXeK0,"Piano concerto No. 2 in G Minor, Op. 22: Piano...",spotify:album:4GunE6Xrwkb93FgXOYoxkP,352160,"Grieg, Saint-Saëns: Piano Concertos",0
...,...,...,...,...,...,...,...,...,...
334408,91,The Velvet Underground,spotify:track:1o6DclZHw0HomxXEQR1ijX,spotify:artist:1nJvji2KIlWSseXRSlNYsC,Rock & Roll - 2015 Remastered Full Length Version,spotify:album:2ny1rPsLOoKNFZhMLubMrs,283000,Loaded: Re-Loaded 45th Anniversary Edition,0
334422,105,The Replacements,spotify:track:0ritgEzDOsxbd2IfYX96S9,spotify:artist:4WPY0N74T3KUja57xMQTZ3,Bastards of Young - 2008 Remastered Version,spotify:album:47arM25GJraK9aWEk4A1Ve,218066,Tim [Expanded Edition],0
334423,106,Violent Femmes,spotify:track:4848zKxd295crunOq6O85I,spotify:artist:0rpMdBzQXf7aYRnu5fDBJy,Gone Daddy Gone - 2002 Remastered Version,spotify:album:2G9onFLGqlMJd1ThYf0vIB,186266,Violent Femmes,0
334436,119,The Smiths,spotify:track:1oA0tDRFjY5xoNDXuMBGGA,spotify:artist:3yY2gUcIsjMr8hjo51PoJ8,"Please, Please, Please Let Me Get What I Want ...",spotify:album:45StnugV9WQMQwk4rRoTy8,112706,Louder Than Bombs,0
